# 2019 NBA Regular Season Evaluation

## Import needed packages and config

In [29]:
import requests
import mysql.connector
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Connect to MySql  and create database. Commented out once DB is created

In [30]:
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
)
cursor = cnx.cursor(buffered=True)

## (re)Connect to database, with database defined, to create table

In [10]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

## Make API Call to grab 2019 NHL reg season data


In [31]:
api_key = config.nba_key
headers = {'Ocp-Apim-Subscription-Key' : '{}'.format(api_key)}

In [26]:
nba_data = []
url = 'https://api.sportsdata.io/v3/nba/scores/json/Games/2019'
response = requests.get(url, headers=headers)
data = response.json()
nba_data.append(data)

In [27]:
nba_data = nba_data[0] # data came back as list of lists of dictionaries.

In [28]:
nba_data

{'statusCode': 401,
 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

In [17]:
parsed_games_nba = []
for game in nba_data:
    parsed_game = {}
    parsed_game['game_id'] = game['GameID']
    parsed_game['month'] = game['DateTime'][5:7]
    parsed_game['away_team'] = game['AwayTeam']
    parsed_game['away_team_score'] = game['AwayTeamScore']
    parsed_game['away_team_moneyline'] = game['PointSpreadAwayTeamMoneyLine']
    parsed_game['home_team'] = game['HomeTeam']
    parsed_game['home_team_score'] = game['HomeTeamScore']
    parsed_game['home_team_moneyline'] = game['PointSpreadHomeTeamMoneyLine']
    parsed_game['spread'] = game['PointSpread']
    parsed_games_nba.append(parsed_game)

In [18]:
parsed_games_nba

[{'game_id': 13093,
  'month': '10',
  'away_team': 'PHI',
  'away_team_score': 9,
  'away_team_moneyline': 242,
  'home_team': 'OTT',
  'home_team_score': 5,
  'home_team_moneyline': -265,
  'spread': 2.0},
 {'game_id': 13094,
  'month': '10',
  'away_team': 'VEG',
  'away_team_score': 3,
  'away_team_moneyline': -242,
  'home_team': 'WAS',
  'home_team_score': 7,
  'home_team_moneyline': 219,
  'spread': -1.7},
 {'game_id': 13095,
  'month': '10',
  'away_team': 'ARI',
  'away_team_score': 4,
  'away_team_moneyline': -236,
  'home_team': 'ANA',
  'home_team_score': 3,
  'home_team_moneyline': 216,
  'spread': -1.7},
 {'game_id': 13096,
  'month': '10',
  'away_team': 'MON',
  'away_team_score': 3,
  'away_team_moneyline': -144,
  'home_team': 'TOR',
  'home_team_score': 4,
  'home_team_moneyline': 132,
  'spread': -1.7},
 {'game_id': 13097,
  'month': '10',
  'away_team': 'BOS',
  'away_team_score': 0,
  'away_team_moneyline': -306,
  'home_team': 'WAS',
  'home_team_score': 9,
  'ho

## Creating NHL Table

In [19]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [ ]:
# cursor.execute("""
# CREATE DATABASE MLB
# """)

# cursor.close()
# cnx.close()

In [20]:
cursor.execute("""
    CREATE TABLE nba (
    game_id varchar(11) NOT NULL primary key,
    month varchar(2) not null,
    away_team varchar(11) not null,
    away_team_score int(3) not null,
    home_team varchar(11) not null,
    home_team_score int(3) not null,
    away_team_moneyline int(4) not null,
    home_team_moneyline int(4) not null,
    spread float(5) not null
    )
""")

## Inserting to table

In [21]:
nba_games_to_insert = []

for game in parsed_games_nba:

    nba_game_to_insert = (
        game['game_id'], 
        game['month'], 
        game['away_team'],
        game['away_team_score'],
        game['home_team'], 
        game['home_team_score'],
        game['away_team_moneyline'],
        game['home_team_moneyline'],
        game['spread'] 
    )
    
    nba_games_to_insert.append(nba_game_to_insert)

In [22]:
insert_query = """
    INSERT INTO nba (
            game_id,
            month,
            away_team,
            away_team_score,
            home_team,
            home_team_score,
            away_team_moneyline,
            home_team_moneyline,
            spread
        )
    VALUES (
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            %s
        )
"""
cursor.executemany(insert_query, nba_games_to_insert)
cnx.commit()

### Questions (Null Hypothesis)

Is there a "home field" advantage(in general , 2018 season)?
    
Will a home team preform better (by month)?

If home team cover more than away team ("good teams win, great teams cover")?


## Creating MLB DataFrame

In [23]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [25]:
cursor.execute(  """SELECT * FROM nba
                    order by game_id;""")
#assigning database (pointed in MySql) as a dataframe using pandas                         
df_nba = pd.DataFrame(cursor.fetchall())


#assigning the column names 
df_nba.columns=['game_id','month', 'away_team', 'away_team_score', 'home_team', 'home_team_score', 'away_team_moneyline', 'home_team_moneyline', 'spread']

In [26]:
df_nba

,game_id,month,away_team,away_team_score,home_team,home_team_score,away_team_moneyline,home_team_moneyline,spread
0,13093,10,PHI,9,OTT,5,242,-265,2.0
1,13094,10,VEG,3,WAS,7,-242,219,-1.7
2,13095,10,ARI,4,ANA,3,-236,216,-1.7
3,13096,10,MON,3,TOR,4,-144,132,-1.7
4,13097,10,BOS,0,WAS,9,-306,283,-1.7
5,13098,10,CGY,3,VAN,7,242,-265,2.0
6,13099,10,ANA,7,SJ,3,-196,184,-1.7
7,13100,10,BOS,5,BUF,0,230,-253,2.0
8,13101,10,NYI,3,CAR,1,-219,201,-1.7
9,13102,10,NAS,4,NYR,3,213,-230,2.0
